In [418]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 
import pyarrow.parquet as pq
import pyarrow as pa
import os

In [419]:
chile_df = pd.read_csv('data/chile.csv',  low_memory=False)
argentina_df = pd.read_csv('data/argentina.csv',  low_memory=False)
antartica_df = pd.read_csv('data/antartica.csv',  low_memory=False)

In [420]:
#info que muestre todos los datos 
chile_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9481 entries, 0 to 9480
Data columns (total 146 columns):
 #    Column               Dtype  
---   ------               -----  
 0    occurrence_no        int64  
 1    record_type          object 
 2    reid_no              float64
 3    flags                object 
 4    collection_no        int64  
 5    accepted_name        object 
 6    accepted_attr        float64
 7    accepted_rank        object 
 8    accepted_no          float64
 9    early_interval       object 
 10   late_interval        object 
 11   max_ma               float64
 12   min_ma               float64
 13   ref_author           object 
 14   ref_pubyr            int64  
 15   reference_no         int64  
 16   phylum               object 
 17   phylum_no            object 
 18   class                object 
 19   class_no             object 
 20   order                object 
 21   order_no             object 
 22   family               object 
 23   family_no  

In [421]:
argentina_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31881 entries, 0 to 31880
Data columns (total 146 columns):
 #    Column               Dtype  
---   ------               -----  
 0    occurrence_no        int64  
 1    record_type          object 
 2    reid_no              float64
 3    flags                object 
 4    collection_no        int64  
 5    accepted_name        object 
 6    accepted_attr        float64
 7    accepted_rank        object 
 8    accepted_no          float64
 9    early_interval       object 
 10   late_interval        object 
 11   max_ma               float64
 12   min_ma               float64
 13   ref_author           object 
 14   ref_pubyr            int64  
 15   reference_no         int64  
 16   phylum               object 
 17   phylum_no            object 
 18   class                object 
 19   class_no             object 
 20   order                object 
 21   order_no             object 
 22   family               object 
 23   family_no

In [422]:
antartica_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11311 entries, 0 to 11310
Data columns (total 146 columns):
 #    Column               Dtype  
---   ------               -----  
 0    occurrence_no        int64  
 1    record_type          object 
 2    reid_no              float64
 3    flags                object 
 4    collection_no        int64  
 5    accepted_name        object 
 6    accepted_attr        float64
 7    accepted_rank        object 
 8    accepted_no          float64
 9    early_interval       object 
 10   late_interval        object 
 11   max_ma               float64
 12   min_ma               float64
 13   ref_author           object 
 14   ref_pubyr            int64  
 15   reference_no         int64  
 16   phylum               object 
 17   phylum_no            object 
 18   class                object 
 19   class_no             object 
 20   order                object 
 21   order_no             object 
 22   family               object 
 23   family_no

Como grupo fuimos viendo manualmente que columnas nos servirian para poder responder las preguntas planteadas en la propuesta inicial, y llegamos a un concenso de elegir las siguientes 19 columnas como base para poder responder estas.

In [423]:
columns = [
    'species_name',
    'occurrence_no',
    'accepted_name',
    'accepted_rank',
    'early_interval',
    'late_interval',
    'max_ma',
    'min_ma',
    'phylum',
    'class',
    'order',
    'family',
    'genus',
    'lat',
    'lng',
    'diet',
    'geogscale',
    'life_habit',
    "environment",
    "taxon_environment",
    "primary_name"
]

#Hacer que los df solo tengan las columnas que necesitamos
chile_df = chile_df[columns].copy()
argentina_df = argentina_df[columns].copy()
antartica_df = antartica_df[columns].copy()

In [424]:
chile_df

,species_name,occurrence_no,accepted_name,accepted_rank,early_interval,late_interval,max_ma,min_ma,phylum,class,order,family,genus,lat,lng,diet,geogscale,life_habit,environment,taxon_environment,primary_name
0,ceciliana,16417,Leionucula,genus,Maastrichtian,NaN,72.10,66.000,Mollusca,Bivalvia,Nuculida,Nuculidae,Leionucula,-37.000000,-73.000000,"deposit feeder, suspension feeder",basin,infaunal,NaN,"coastal,inner shelf,outer shelf,oceanic",Leionucula
1,amuriensis,16418,Nuculana,genus,Maastrichtian,NaN,72.10,66.000,Mollusca,Bivalvia,Nuculanida,Nuculanidae,Nuculana,-37.000000,-73.000000,"deposit feeder, suspension feeder",basin,infaunal,NaN,"coastal,inner shelf,outer shelf,oceanic",Nuculana
2,cuneiformis,16419,Nuculana,genus,Maastrichtian,NaN,72.10,66.000,Mollusca,Bivalvia,Nuculanida,Nuculanidae,Nuculana,-37.000000,-73.000000,"deposit feeder, suspension feeder",basin,infaunal,NaN,"coastal,inner shelf,outer shelf,oceanic",Nuculana
3,levitestata,16420,Yoldia,genus,Maastrichtian,NaN,72.10,66.000,Mollusca,Bivalvia,Nuculanida,Yoldiidae,Yoldia,-37.000000,-73.000000,deposit feeder,basin,infaunal,NaN,"inner shelf,outer shelf,oceanic",Yoldia
4,pencana,16421,Neiloides,genus,Maastrichtian,NaN,72.10,66.000,Mollusca,Bivalvia,Nuculida,NO_FAMILY_SPECIFIED,Neiloides,-37.000000,-73.000000,deposit feeder,basin,infaunal,NaN,NaN,Neiloides
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9476,sp.,1659402,Ostrea,genus,Turonian,Campanian,93.90,72.100,Mollusca,Bivalvia,Ostreida,Ostreidae,Ostrea,-50.884998,-72.625000,suspension feeder,NaN,epifaunal,basinal (siliciclastic),"hypersaline,marine,brackish",Ostrea
9477,antarctica,1659403,Cucullaea antarctica,species,Turonian,Campanian,93.90,72.100,Mollusca,Bivalvia,Arcida,Cucullaeidae,Cucullaea,-50.884998,-72.625000,suspension feeder,NaN,infaunal,basinal (siliciclastic),NaN,Cucullaea
9478,sp.,1659404,Nucula,genus,Turonian,Campanian,93.90,72.100,Mollusca,Bivalvia,Nuculida,Nuculidae,Nucula,-50.884998,-72.625000,"deposit feeder, suspension feeder",NaN,infaunal,basinal (siliciclastic),"coastal,inner shelf,outer shelf,oceanic",Nucula
9479,saladensis,1660104,Micrauchenia saladensis,species,Tortonian,Messinian,11.63,5.333,Chordata,Mammalia,Panameriungulata,Macraucheniidae,Micrauchenia,-27.684723,-70.949997,herbivore,NaN,scansorial,coastal indet.,terrestrial,Micrauchenia


In [425]:
obtener_duplicados_chile = chile_df.duplicated(keep=False)
datos_duplicados_chile = chile_df[obtener_duplicados_chile]
datos_duplicados_chile

,species_name,occurrence_no,accepted_name,accepted_rank,early_interval,late_interval,max_ma,min_ma,phylum,class,order,family,genus,lat,lng,diet,geogscale,life_habit,environment,taxon_environment,primary_name


In [426]:
obtener_duplicados_argentina = chile_df.duplicated(keep=False)
datos_duplicados_argentina = chile_df[obtener_duplicados_argentina]
datos_duplicados_argentina

,species_name,occurrence_no,accepted_name,accepted_rank,early_interval,late_interval,max_ma,min_ma,phylum,class,order,family,genus,lat,lng,diet,geogscale,life_habit,environment,taxon_environment,primary_name


In [427]:
obtener_duplicados_antartica = antartica_df.duplicated(keep=False)
datos_duplicados_antartica = antartica_df[obtener_duplicados_antartica]
datos_duplicados_antartica

,species_name,occurrence_no,accepted_name,accepted_rank,early_interval,late_interval,max_ma,min_ma,phylum,class,order,family,genus,lat,lng,diet,geogscale,life_habit,environment,taxon_environment,primary_name


In [428]:
chile_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9481 entries, 0 to 9480
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species_name       9481 non-null   object 
 1   occurrence_no      9481 non-null   int64  
 2   accepted_name      9343 non-null   object 
 3   accepted_rank      9343 non-null   object 
 4   early_interval     9481 non-null   object 
 5   late_interval      1605 non-null   object 
 6   max_ma             9481 non-null   float64
 7   min_ma             9481 non-null   float64
 8   phylum             9337 non-null   object 
 9   class              9292 non-null   object 
 10  order              9123 non-null   object 
 11  family             8914 non-null   object 
 12  genus              8574 non-null   object 
 13  lat                9481 non-null   float64
 14  lng                9481 non-null   float64
 15  diet               8749 non-null   object 
 16  geogscale          6426 

In [429]:
argentina_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31881 entries, 0 to 31880
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species_name       31881 non-null  object 
 1   occurrence_no      31881 non-null  int64  
 2   accepted_name      31253 non-null  object 
 3   accepted_rank      31253 non-null  object 
 4   early_interval     31881 non-null  object 
 5   late_interval      5288 non-null   object 
 6   max_ma             31881 non-null  float64
 7   min_ma             31881 non-null  float64
 8   phylum             31226 non-null  object 
 9   class              31038 non-null  object 
 10  order              29738 non-null  object 
 11  family             29056 non-null  object 
 12  genus              27398 non-null  object 
 13  lat                31881 non-null  float64
 14  lng                31881 non-null  float64
 15  diet               24717 non-null  object 
 16  geogscale          256

In [430]:
antartica_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11311 entries, 0 to 11310
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species_name       11311 non-null  object 
 1   occurrence_no      11311 non-null  int64  
 2   accepted_name      10977 non-null  object 
 3   accepted_rank      10977 non-null  object 
 4   early_interval     11311 non-null  object 
 5   late_interval      2888 non-null   object 
 6   max_ma             11311 non-null  float64
 7   min_ma             11311 non-null  float64
 8   phylum             10960 non-null  object 
 9   class              10926 non-null  object 
 10  order              10744 non-null  object 
 11  family             10604 non-null  object 
 12  genus              10375 non-null  object 
 13  lat                11311 non-null  float64
 14  lng                11311 non-null  float64
 15  diet               7706 non-null   object 
 16  geogscale          828

In [431]:
chile_df['era_early'] = None
argentina_df['era_early'] = None
antartica_df['era_early'] = None

In [432]:
chile_df['era_late'] = None
argentina_df['era_late'] = None
antartica_df['era_late'] = None

In [433]:
# Listas de intervalos para cada época
cambrian_intervals = [
    'Terreneuvian', 'Series 2', 'Miaolingian', 'Furongian', 'Fortunian',
    'Stage 2', 'Stage 3', 'Stage 4', 'Wuliuan', 'Drumian', 'Guzhangian', 
    'Paibian', 'Jiangshanian', 'Stage 10'
]

silurian_intervals = [
    'Rhuddanian', 'Aeronian', 'Telychian', 'Sheinwoodian', 'Homerian', 
    'Gorstian', 'Ludfordian', 'Llandovery', 'Wenlock', 'Ludlow', 'Pridoli'
]

devonian_intervals = [
    'Lochkovian', 'Pragian', 'Emsian', 'Eifelian', 'Givetian', 'Fransnian', 
    'Famennian', 'Early Devonian', 'Middle Devonian', 'Late Devonian'
]

carboniferous_intervals = [
    'Tournaisian', 'Visean', 'Serpukhovian', 'Bashkirian', 'Moscovian', 
    'Kasimovian', 'Gzhelian', 'Mississippian', 'Pennsylvanian'
]

permian_intervals = [
    'Asselian', 'Sakmarian', 'Artinskian', 'Kungurian', 'Roadian', 'Wordian', 
    'Capitanian', 'Wuchiapingian', 'Changhsingian', 'Cisuralian', 'Guadalupian', 
    'Lopingian'
]

# Agregar los intervalos para el Ordovícico
ordovician_intervals = [
    'Early Ordovician', 'Middle Ordovician', 'Late Ordovician', 
    'Tremadocian', 'Floian', 'Dapingian', 'Darriwilian', 
    'Sandbian', 'Katian', 'Hirnantian'
]
triassic_intervals = [
    'Induan', 'Anisian', 'Ladinian', 'Carnian', 'Norian', 'Rhaetian', 
    'Early Triassic', 'Middle Triassic', 'Late Triassic'
]

neogene_intervals = [
    'Miocene', 'Pliocene', 'Aquitanian', 'Burdigalian', 'Langhian', 'Serrevallian', 
    'Tortonian', 'Messinian', 'Zanclean', 'Piacenzian'
]

quaternary_intervals = [
    'Pleistocene', 'Gelasian', 'Calabrian', 'Chibanian'
]

# Agregar los intervalos para el Jurásico
jurassic_intervals = [
    'Hettangian', 'Sineamurian', 'Pliensbachian', 'Toarcian', 'Aalenain', 
    'Bajocian', 'Bathonian', 'Callovian', 'Oxfordian', 'Kimmeridgian', 
    'Tithonian', 'Early Jurassic', 'Middle Jurassic', 'Late Jurassic'
]

paleogene_intervals = [
    'Paleocene', 'Eocene', 'Oligocene', 'Danian', 'Selandian', 'Thanetian', 
    'Ypresian', 'Lutetian', 'Bartonian', 'Priabonian', 'Rupelian', 'Chattian'
]

# Agregar los intervalos para el Cretácico
cretaceous_intervals = [
    'Berriasian', 'Valanginian', 'Hauterivian', 'Barremian', 'Aptian', 'Albian', 
    'Cenomanian', 'Turonian', 'Coniacian', 'Santonian', 'Campanian', 'Maastrichtian', 
    'Early Cretaceous', 'Late Cretaceous'
]
# Función para asignar la era_early según el valor de 'early_interval'
def assign_era(row):
    if row['early_interval'] in cambrian_intervals:
        return 'Cambrian'
    elif row['early_interval'] in silurian_intervals:
        return 'Silurian'
    elif row['early_interval'] in devonian_intervals:
        return 'Devonian'
    elif row['early_interval'] in carboniferous_intervals:
        return 'Carboniferous'
    elif row['early_interval'] in permian_intervals:
        return 'Permian'
    elif row['early_interval'] in ordovician_intervals:
        return 'Ordovician'  # Añadimos la condición para Ordovícico
    elif row['early_interval'] in triassic_intervals:
        return 'Triassic'  # Añadimos la condición para Triásico
    elif row['early_interval'] in neogene_intervals:
        return 'Neogene'
    elif row['early_interval'] in quaternary_intervals:  # Añadimos la condición para Cuaternario
        return 'Quaternary'
    elif row['early_interval'] in jurassic_intervals:  # Añadimos la condición para Jurásico
        return 'Jurassic'
    elif row['early_interval'] in paleogene_intervals:  # Añadimos la condición para Paleógeno
        return 'Paleogene'
    elif row['early_interval'] in cretaceous_intervals:  # Añadimos la condición para Cretácico
        return 'Cretaceous'
    else:
        return row['early_interval']

# Aplicar la función a cada fila
chile_df['era_early'] = chile_df.apply(assign_era, axis=1)

In [434]:
# Listas de intervalos para cada época
cambrian_intervals = [
    'Terreneuvian', 'Series 2', 'Miaolingian', 'Furongian', 'Fortunian',
    'Stage 2', 'Stage 3', 'Stage 4', 'Wuliuan', 'Drumian', 'Guzhangian', 
    'Paibian', 'Jiangshanian', 'Stage 10'
]

silurian_intervals = [
    'Rhuddanian', 'Aeronian', 'Telychian', 'Sheinwoodian', 'Homerian', 
    'Gorstian', 'Ludfordian', 'Llandovery', 'Wenlock', 'Ludlow', 'Pridoli'
]

devonian_intervals = [
    'Lochkovian', 'Pragian', 'Emsian', 'Eifelian', 'Givetian', 'Fransnian', 
    'Famennian', 'Early Devonian', 'Middle Devonian', 'Late Devonian'
]

carboniferous_intervals = [
    'Tournaisian', 'Visean', 'Serpukhovian', 'Bashkirian', 'Moscovian', 
    'Kasimovian', 'Gzhelian', 'Mississippian', 'Pennsylvanian'
]

permian_intervals = [
    'Asselian', 'Sakmarian', 'Artinskian', 'Kungurian', 'Roadian', 'Wordian', 
    'Capitanian', 'Wuchiapingian', 'Changhsingian', 'Cisuralian', 'Guadalupian', 
    'Lopingian'
]

# Agregar los intervalos para el Ordovícico
ordovician_intervals = [
    'Early Ordovician', 'Middle Ordovician', 'Late Ordovician', 
    'Tremadocian', 'Floian', 'Dapingian', 'Darriwilian', 
    'Sandbian', 'Katian', 'Hirnantian'
]
triassic_intervals = [
    'Induan', 'Anisian', 'Ladinian', 'Carnian', 'Norian', 'Rhaetian', 
    'Early Triassic', 'Middle Triassic', 'Late Triassic'
]

neogene_intervals = [
    'Miocene', 'Pliocene', 'Aquitanian', 'Burdigalian', 'Langhian', 'Serrevallian', 
    'Tortonian', 'Messinian', 'Zanclean', 'Piacenzian'
]

quaternary_intervals = [
    'Pleistocene', 'Gelasian', 'Calabrian', 'Chibanian'
]

# Agregar los intervalos para el Jurásico
jurassic_intervals = [
    'Hettangian', 'Sineamurian', 'Pliensbachian', 'Toarcian', 'Aalenain', 
    'Bajocian', 'Bathonian', 'Callovian', 'Oxfordian', 'Kimmeridgian', 
    'Tithonian', 'Early Jurassic', 'Middle Jurassic', 'Late Jurassic'
]

paleogene_intervals = [
    'Paleocene', 'Eocene', 'Oligocene', 'Danian', 'Selandian', 'Thanetian', 
    'Ypresian', 'Lutetian', 'Bartonian', 'Priabonian', 'Rupelian', 'Chattian'
]

# Agregar los intervalos para el Cretácico
cretaceous_intervals = [
    'Berriasian', 'Valanginian', 'Hauterivian', 'Barremian', 'Aptian', 'Albian', 
    'Cenomanian', 'Turonian', 'Coniacian', 'Santonian', 'Campanian', 'Maastrichtian', 
    'Early Cretaceous', 'Late Cretaceous'
]
# Función para asignar la era_early según el valor de 'early_interval'
def assign_era(row):
    if row['early_interval'] in cambrian_intervals:
        return 'Cambrian'
    elif row['early_interval'] in silurian_intervals:
        return 'Silurian'
    elif row['early_interval'] in devonian_intervals:
        return 'Devonian'
    elif row['early_interval'] in carboniferous_intervals:
        return 'Carboniferous'
    elif row['early_interval'] in permian_intervals:
        return 'Permian'
    elif row['early_interval'] in ordovician_intervals:
        return 'Ordovician'  # Añadimos la condición para Ordovícico
    elif row['early_interval'] in triassic_intervals:
        return 'Triassic'  # Añadimos la condición para Triásico
    elif row['early_interval'] in neogene_intervals:
        return 'Neogene'
    elif row['early_interval'] in quaternary_intervals:  # Añadimos la condición para Cuaternario
        return 'Quaternary'
    elif row['early_interval'] in jurassic_intervals:  # Añadimos la condición para Jurásico
        return 'Jurassic'
    elif row['early_interval'] in paleogene_intervals:  # Añadimos la condición para Paleógeno
        return 'Paleogene'
    elif row['early_interval'] in cretaceous_intervals:  # Añadimos la condición para Cretácico
        return 'Cretaceous'
    else:
        return row['early_interval']

# Aplicar la función a cada fila
argentina_df['era_early'] = argentina_df.apply(assign_era, axis=1)

In [435]:
# Listas de intervalos para cada época
cambrian_intervals = [
    'Terreneuvian', 'Series 2', 'Miaolingian', 'Furongian', 'Fortunian',
    'Stage 2', 'Stage 3', 'Stage 4', 'Wuliuan', 'Drumian', 'Guzhangian', 
    'Paibian', 'Jiangshanian', 'Stage 10'
]

silurian_intervals = [
    'Rhuddanian', 'Aeronian', 'Telychian', 'Sheinwoodian', 'Homerian', 
    'Gorstian', 'Ludfordian', 'Llandovery', 'Wenlock', 'Ludlow', 'Pridoli'
]

devonian_intervals = [
    'Lochkovian', 'Pragian', 'Emsian', 'Eifelian', 'Givetian', 'Fransnian', 
    'Famennian', 'Early Devonian', 'Middle Devonian', 'Late Devonian'
]

carboniferous_intervals = [
    'Tournaisian', 'Visean', 'Serpukhovian', 'Bashkirian', 'Moscovian', 
    'Kasimovian', 'Gzhelian', 'Mississippian', 'Pennsylvanian'
]

permian_intervals = [
    'Asselian', 'Sakmarian', 'Artinskian', 'Kungurian', 'Roadian', 'Wordian', 
    'Capitanian', 'Wuchiapingian', 'Changhsingian', 'Cisuralian', 'Guadalupian', 
    'Lopingian'
]

# Agregar los intervalos para el Ordovícico
ordovician_intervals = [
    'Early Ordovician', 'Middle Ordovician', 'Late Ordovician', 
    'Tremadocian', 'Floian', 'Dapingian', 'Darriwilian', 
    'Sandbian', 'Katian', 'Hirnantian'
]
triassic_intervals = [
    'Induan', 'Anisian', 'Ladinian', 'Carnian', 'Norian', 'Rhaetian', 
    'Early Triassic', 'Middle Triassic', 'Late Triassic'
]

neogene_intervals = [
    'Miocene', 'Pliocene', 'Aquitanian', 'Burdigalian', 'Langhian', 'Serrevallian', 
    'Tortonian', 'Messinian', 'Zanclean', 'Piacenzian'
]

quaternary_intervals = [
    'Pleistocene', 'Gelasian', 'Calabrian', 'Chibanian'
]

# Agregar los intervalos para el Jurásico
jurassic_intervals = [
    'Hettangian', 'Sineamurian', 'Pliensbachian', 'Toarcian', 'Aalenain', 
    'Bajocian', 'Bathonian', 'Callovian', 'Oxfordian', 'Kimmeridgian', 
    'Tithonian', 'Early Jurassic', 'Middle Jurassic', 'Late Jurassic'
]

paleogene_intervals = [
    'Paleocene', 'Eocene', 'Oligocene', 'Danian', 'Selandian', 'Thanetian', 
    'Ypresian', 'Lutetian', 'Bartonian', 'Priabonian', 'Rupelian', 'Chattian'
]

# Agregar los intervalos para el Cretácico
cretaceous_intervals = [
    'Berriasian', 'Valanginian', 'Hauterivian', 'Barremian', 'Aptian', 'Albian', 
    'Cenomanian', 'Turonian', 'Coniacian', 'Santonian', 'Campanian', 'Maastrichtian', 
    'Early Cretaceous', 'Late Cretaceous'
]
# Función para asignar la era_early según el valor de 'early_interval'
def assign_era(row):
    if row['early_interval'] in cambrian_intervals:
        return 'Cambrian'
    elif row['early_interval'] in silurian_intervals:
        return 'Silurian'
    elif row['early_interval'] in devonian_intervals:
        return 'Devonian'
    elif row['early_interval'] in carboniferous_intervals:
        return 'Carboniferous'
    elif row['early_interval'] in permian_intervals:
        return 'Permian'
    elif row['early_interval'] in ordovician_intervals:
        return 'Ordovician'  # Añadimos la condición para Ordovícico
    elif row['early_interval'] in triassic_intervals:
        return 'Triassic'  # Añadimos la condición para Triásico
    elif row['early_interval'] in neogene_intervals:
        return 'Neogene'
    elif row['early_interval'] in quaternary_intervals:  # Añadimos la condición para Cuaternario
        return 'Quaternary'
    elif row['early_interval'] in jurassic_intervals:  # Añadimos la condición para Jurásico
        return 'Jurassic'
    elif row['early_interval'] in paleogene_intervals:  # Añadimos la condición para Paleógeno
        return 'Paleogene'
    elif row['early_interval'] in cretaceous_intervals:  # Añadimos la condición para Cretácico
        return 'Cretaceous'
    else:
        return row['early_interval']

# Aplicar la función a cada fila
antartica_df['era_early'] = antartica_df.apply(assign_era, axis=1)

In [436]:
# Listas de intervalos para cada época
cambrian_intervals = [
    'Terreneuvian', 'Series 2', 'Miaolingian', 'Furongian', 'Fortunian',
    'Stage 2', 'Stage 3', 'Stage 4', 'Wuliuan', 'Drumian', 'Guzhangian', 
    'Paibian', 'Jiangshanian', 'Stage 10'
]

silurian_intervals = [
    'Rhuddanian', 'Aeronian', 'Telychian', 'Sheinwoodian', 'Homerian', 
    'Gorstian', 'Ludfordian', 'Llandovery', 'Wenlock', 'Ludlow', 'Pridoli'
]

devonian_intervals = [
    'Lochkovian', 'Pragian', 'Emsian', 'Eifelian', 'Givetian', 'Fransnian', 
    'Famennian', 'Early Devonian', 'Middle Devonian', 'Late Devonian'
]

carboniferous_intervals = [
    'Tournaisian', 'Visean', 'Serpukhovian', 'Bashkirian', 'Moscovian', 
    'Kasimovian', 'Gzhelian', 'Mississippian', 'Pennsylvanian'
]

permian_intervals = [
    'Asselian', 'Sakmarian', 'Artinskian', 'Kungurian', 'Roadian', 'Wordian', 
    'Capitanian', 'Wuchiapingian', 'Changhsingian', 'Cisuralian', 'Guadalupian', 
    'Lopingian'
]

# Agregar los intervalos para el Ordovícico
ordovician_intervals = [
    'Early Ordovician', 'Middle Ordovician', 'Late Ordovician', 
    'Tremadocian', 'Floian', 'Dapingian', 'Darriwilian', 
    'Sandbian', 'Katian', 'Hirnantian'
]
triassic_intervals = [
    'Induan', 'Anisian', 'Ladinian', 'Carnian', 'Norian', 'Rhaetian', 
    'Early Triassic', 'Middle Triassic', 'Late Triassic'
]

neogene_intervals = [
    'Miocene', 'Pliocene', 'Aquitanian', 'Burdigalian', 'Langhian', 'Serrevallian', 
    'Tortonian', 'Messinian', 'Zanclean', 'Piacenzian'
]

quaternary_intervals = [
    'Pleistocene', 'Gelasian', 'Calabrian', 'Chibanian'
]

# Agregar los intervalos para el Jurásico
jurassic_intervals = [
    'Hettangian', 'Sineamurian', 'Pliensbachian', 'Toarcian', 'Aalenain', 
    'Bajocian', 'Bathonian', 'Callovian', 'Oxfordian', 'Kimmeridgian', 
    'Tithonian', 'Early Jurassic', 'Middle Jurassic', 'Late Jurassic'
]

paleogene_intervals = [
    'Paleocene', 'Eocene', 'Oligocene', 'Danian', 'Selandian', 'Thanetian', 
    'Ypresian', 'Lutetian', 'Bartonian', 'Priabonian', 'Rupelian', 'Chattian'
]

# Agregar los intervalos para el Cretácico
cretaceous_intervals = [
    'Berriasian', 'Valanginian', 'Hauterivian', 'Barremian', 'Aptian', 'Albian', 
    'Cenomanian', 'Turonian', 'Coniacian', 'Santonian', 'Campanian', 'Maastrichtian', 
    'Early Cretaceous', 'Late Cretaceous'
]
nan = [np.nan]
# Función para asignar la era_late según el valor de 'late_interval'
def assign_era(row):
    if row['late_interval'] in cambrian_intervals:
        return 'Cambrian'
    elif row['late_interval'] in silurian_intervals:
        return 'Silurian'
    elif row['late_interval'] in devonian_intervals:
        return 'Devonian'
    elif row['late_interval'] in carboniferous_intervals:
        return 'Carboniferous'
    elif row['late_interval'] in permian_intervals:
        return 'Permian'
    elif row['late_interval'] in ordovician_intervals:
        return 'Ordovician'  
    elif row['late_interval'] in triassic_intervals:
        return 'Triassic'  
    elif row['late_interval'] in neogene_intervals:
        return 'Neogene'
    elif row['late_interval'] in quaternary_intervals:  
        return 'Quaternary'
    elif row['late_interval'] in jurassic_intervals:  
        return 'Jurassic'
    elif row['late_interval'] in paleogene_intervals:  
        return 'Paleogene'
    elif row['late_interval'] in cretaceous_intervals:  
        return 'Cretaceous'
    elif row['late_interval'] in nan:
        return "No especificado"
    else:
        return row['early_interval']

# Aplicar la función a cada fila
chile_df['era_late'] = chile_df.apply(assign_era, axis=1)

In [437]:
# Listas de intervalos para cada época
cambrian_intervals = [
    'Terreneuvian', 'Series 2', 'Miaolingian', 'Furongian', 'Fortunian',
    'Stage 2', 'Stage 3', 'Stage 4', 'Wuliuan', 'Drumian', 'Guzhangian', 
    'Paibian', 'Jiangshanian', 'Stage 10'
]

silurian_intervals = [
    'Rhuddanian', 'Aeronian', 'Telychian', 'Sheinwoodian', 'Homerian', 
    'Gorstian', 'Ludfordian', 'Llandovery', 'Wenlock', 'Ludlow', 'Pridoli'
]

devonian_intervals = [
    'Lochkovian', 'Pragian', 'Emsian', 'Eifelian', 'Givetian', 'Fransnian', 
    'Famennian', 'Early Devonian', 'Middle Devonian', 'Late Devonian'
]

carboniferous_intervals = [
    'Tournaisian', 'Visean', 'Serpukhovian', 'Bashkirian', 'Moscovian', 
    'Kasimovian', 'Gzhelian', 'Mississippian', 'Pennsylvanian'
]

permian_intervals = [
    'Asselian', 'Sakmarian', 'Artinskian', 'Kungurian', 'Roadian', 'Wordian', 
    'Capitanian', 'Wuchiapingian', 'Changhsingian', 'Cisuralian', 'Guadalupian', 
    'Lopingian'
]

# Agregar los intervalos para el Ordovícico
ordovician_intervals = [
    'Early Ordovician', 'Middle Ordovician', 'Late Ordovician', 
    'Tremadocian', 'Floian', 'Dapingian', 'Darriwilian', 
    'Sandbian', 'Katian', 'Hirnantian'
]
triassic_intervals = [
    'Induan', 'Anisian', 'Ladinian', 'Carnian', 'Norian', 'Rhaetian', 
    'Early Triassic', 'Middle Triassic', 'Late Triassic'
]

neogene_intervals = [
    'Miocene', 'Pliocene', 'Aquitanian', 'Burdigalian', 'Langhian', 'Serrevallian', 
    'Tortonian', 'Messinian', 'Zanclean', 'Piacenzian'
]

quaternary_intervals = [
    'Pleistocene', 'Gelasian', 'Calabrian', 'Chibanian'
]

# Agregar los intervalos para el Jurásico
jurassic_intervals = [
    'Hettangian', 'Sineamurian', 'Pliensbachian', 'Toarcian', 'Aalenain', 
    'Bajocian', 'Bathonian', 'Callovian', 'Oxfordian', 'Kimmeridgian', 
    'Tithonian', 'Early Jurassic', 'Middle Jurassic', 'Late Jurassic'
]

paleogene_intervals = [
    'Paleocene', 'Eocene', 'Oligocene', 'Danian', 'Selandian', 'Thanetian', 
    'Ypresian', 'Lutetian', 'Bartonian', 'Priabonian', 'Rupelian', 'Chattian'
]

# Agregar los intervalos para el Cretácico
cretaceous_intervals = [
    'Berriasian', 'Valanginian', 'Hauterivian', 'Barremian', 'Aptian', 'Albian', 
    'Cenomanian', 'Turonian', 'Coniacian', 'Santonian', 'Campanian', 'Maastrichtian', 
    'Early Cretaceous', 'Late Cretaceous'
]
nan = [np.nan]
# Función para asignar la era_late según el valor de 'late_interval'
def assign_era(row):
    if row['late_interval'] in cambrian_intervals:
        return 'Cambrian'
    elif row['late_interval'] in silurian_intervals:
        return 'Silurian'
    elif row['late_interval'] in devonian_intervals:
        return 'Devonian'
    elif row['late_interval'] in carboniferous_intervals:
        return 'Carboniferous'
    elif row['late_interval'] in permian_intervals:
        return 'Permian'
    elif row['late_interval'] in ordovician_intervals:
        return 'Ordovician'  
    elif row['late_interval'] in triassic_intervals:
        return 'Triassic'  
    elif row['late_interval'] in neogene_intervals:
        return 'Neogene'
    elif row['late_interval'] in quaternary_intervals:  
        return 'Quaternary'
    elif row['late_interval'] in jurassic_intervals:  
        return 'Jurassic'
    elif row['late_interval'] in paleogene_intervals:  
        return 'Paleogene'
    elif row['late_interval'] in cretaceous_intervals:  
        return 'Cretaceous'
    elif row['late_interval'] in nan:
        return "No especificado"
    else:
        return row['early_interval']

# Aplicar la función a cada fila
argentina_df['era_late'] = argentina_df.apply(assign_era, axis=1)

In [438]:
# Listas de intervalos para cada época
cambrian_intervals = [
    'Terreneuvian', 'Series 2', 'Miaolingian', 'Furongian', 'Fortunian',
    'Stage 2', 'Stage 3', 'Stage 4', 'Wuliuan', 'Drumian', 'Guzhangian', 
    'Paibian', 'Jiangshanian', 'Stage 10'
]

silurian_intervals = [
    'Rhuddanian', 'Aeronian', 'Telychian', 'Sheinwoodian', 'Homerian', 
    'Gorstian', 'Ludfordian', 'Llandovery', 'Wenlock', 'Ludlow', 'Pridoli'
]

devonian_intervals = [
    'Lochkovian', 'Pragian', 'Emsian', 'Eifelian', 'Givetian', 'Fransnian', 
    'Famennian', 'Early Devonian', 'Middle Devonian', 'Late Devonian'
]

carboniferous_intervals = [
    'Tournaisian', 'Visean', 'Serpukhovian', 'Bashkirian', 'Moscovian', 
    'Kasimovian', 'Gzhelian', 'Mississippian', 'Pennsylvanian'
]

permian_intervals = [
    'Asselian', 'Sakmarian', 'Artinskian', 'Kungurian', 'Roadian', 'Wordian', 
    'Capitanian', 'Wuchiapingian', 'Changhsingian', 'Cisuralian', 'Guadalupian', 
    'Lopingian'
]

# Agregar los intervalos para el Ordovícico
ordovician_intervals = [
    'Early Ordovician', 'Middle Ordovician', 'Late Ordovician', 
    'Tremadocian', 'Floian', 'Dapingian', 'Darriwilian', 
    'Sandbian', 'Katian', 'Hirnantian'
]
triassic_intervals = [
    'Induan', 'Anisian', 'Ladinian', 'Carnian', 'Norian', 'Rhaetian', 
    'Early Triassic', 'Middle Triassic', 'Late Triassic'
]

neogene_intervals = [
    'Miocene', 'Pliocene', 'Aquitanian', 'Burdigalian', 'Langhian', 'Serrevallian', 
    'Tortonian', 'Messinian', 'Zanclean', 'Piacenzian'
]

quaternary_intervals = [
    'Pleistocene', 'Gelasian', 'Calabrian', 'Chibanian'
]

# Agregar los intervalos para el Jurásico
jurassic_intervals = [
    'Hettangian', 'Sineamurian', 'Pliensbachian', 'Toarcian', 'Aalenain', 
    'Bajocian', 'Bathonian', 'Callovian', 'Oxfordian', 'Kimmeridgian', 
    'Tithonian', 'Early Jurassic', 'Middle Jurassic', 'Late Jurassic'
]

paleogene_intervals = [
    'Paleocene', 'Eocene', 'Oligocene', 'Danian', 'Selandian', 'Thanetian', 
    'Ypresian', 'Lutetian', 'Bartonian', 'Priabonian', 'Rupelian', 'Chattian'
]

# Agregar los intervalos para el Cretácico
cretaceous_intervals = [
    'Berriasian', 'Valanginian', 'Hauterivian', 'Barremian', 'Aptian', 'Albian', 
    'Cenomanian', 'Turonian', 'Coniacian', 'Santonian', 'Campanian', 'Maastrichtian', 
    'Early Cretaceous', 'Late Cretaceous'
]
nan = [np.nan]
# Función para asignar la era_late según el valor de 'late_interval'
def assign_era(row):
    if row['late_interval'] in cambrian_intervals:
        return 'Cambrian'
    elif row['late_interval'] in silurian_intervals:
        return 'Silurian'
    elif row['late_interval'] in devonian_intervals:
        return 'Devonian'
    elif row['late_interval'] in carboniferous_intervals:
        return 'Carboniferous'
    elif row['late_interval'] in permian_intervals:
        return 'Permian'
    elif row['late_interval'] in ordovician_intervals:
        return 'Ordovician'  
    elif row['late_interval'] in triassic_intervals:
        return 'Triassic'  
    elif row['late_interval'] in neogene_intervals:
        return 'Neogene'
    elif row['late_interval'] in quaternary_intervals:  
        return 'Quaternary'
    elif row['late_interval'] in jurassic_intervals:  
        return 'Jurassic'
    elif row['late_interval'] in paleogene_intervals:  
        return 'Paleogene'
    elif row['late_interval'] in cretaceous_intervals:  
        return 'Cretaceous'
    elif row['late_interval'] in nan:
        return "No especificado"
    else:
        return row['early_interval']

# Aplicar la función a cada fila
antartica_df['era_late'] = antartica_df.apply(assign_era, axis=1)